In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2

D:\ANACONDA\ANA\envs\aiworks\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\ANACONDA\ANA\envs\aiworks\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
D:\ANACONDA\ANA\envs\aiworks\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def detect_and_predict_emotion(frame, face_cascade, emotionNet):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    locs = []
    preds = []

    for (x, y, w, h) in faces:
        face = gray[y:y + h, x:x + w]
        face = cv2.resize(face, (48, 48))  # Resize face image to 48x48
        face = face.astype("float") / 255.0
        face = img_to_array(face)
        face = np.expand_dims(face, axis=0)

        preds = emotionNet.predict(face)[0]
        locs.append((x, y, x + w, y + h))

    return (locs, preds)


In [3]:
import cv2
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import time  
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
emotionNet = load_model("face_emotion_recognition_model.h5")

print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

EMOTIONS = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

[INFO] starting video stream...


In [4]:
import imutils
import cv2
from tensorflow.keras.models import load_model
import numpy as np

# Load the face cascade and emotionNet model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
emotionNet = load_model("face_emotion_recognition_model.h5")

# Define the list of emotions
EMOTIONS = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

# Start the video stream
vs = cv2.VideoCapture(0)

while True:
    ret, frame = vs.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=400)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    (locs, preds) = detect_and_predict_emotion(frame, face_cascade, emotionNet)

    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box

        emotion_label = EMOTIONS[np.argmax(pred)]
        emotion_probability = np.max(pred)

        label = "{}: {:.2f}%".format(emotion_label, emotion_probability * 100)
        color = (0, 255, 0) if emotion_label == "happy" else (0, 0, 255)

        cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.release()

1/1 [==============================] - 0s 96ms/step


1/1 [==============================] - 0s 104ms/step


1/1 [==============================] - 0s 104ms/step


KeyboardInterrupt: 